# Creating a Team

## RoundRobinGroupChat  

**RoundRobinGroupChat** is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.  

## Team Setup  

We will begin by creating a team with:  
- **Two AssistantAgent instances**  
- A **TextMentionTermination** condition that stops the team when a specific word is detected in an agent’s response.


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

# Create the BlogPostDraftAgent
blog_post_draft_agent = AssistantAgent(
    "BlogPostDraftAgent",
    model_client=model_client,
    system_message="You are a blog post drafting agent. Your task is to generate an outline or initial draft of a blog post or article based on the topic provided. Make sure to include clear sections, headings, and a logical flow."
)

# Create the BlogPostEnhancementAgent
blog_post_enhancement_agent = AssistantAgent(
    "BlogPostEnhancementAgent",
    model_client=model_client,
    system_message="You are a blog post enhancement agent. Your task is to improve the draft provided by the drafting agent. Focus on improving readability, adding relevant examples, and optimizing for SEO. If you are satisfied with the final draft, respond with 'FINAL'."
)

# Define the termination condition that stops the task when 'FINAL' is mentioned
text_termination = TextMentionTermination("FINAL")

# Create a team where the BlogPostDraftAgent and BlogPostEnhancementAgent interact.
team = RoundRobinGroupChat(
    [blog_post_draft_agent, blog_post_enhancement_agent],
    termination_condition=text_termination
)

## Running and Observing a team

In [ ]:
# You can run a team using team.run() or team.run_stream()
async for message in team.run_stream(task="Create a short blog post about the benefits of using AI in e-commerce."): 
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

<br> 
The Console() method provides a convenient way to print messages to the console with proper formatting.

## Resetting a Team

You can reset the team by calling the `reset()` method. This method will clear the team’s state, including all agents. It will invoke each agent’s `on_reset()` method to reset the agent’s state.


In [ ]:

await team.reset()  # Reset the team for the next run.

In [ ]:
await Console(team.run_stream(task="Create a short blog post about the benefits of using AI in e-commerce."))  # Stream the messages to the console.